# NER and Classification with LLMs

The exercise demonstrates how to leverage Large Language Models for NER and text classification tasks, comparing their performance with traditional approaches.


## Tasks

1. Install and configure [OLLama](https://ollama.com/) with an appropriate LLM model (e.g. models from: Llama, Mistral, Bielik, Phi families). Rather not use models above 10B paramters.
Sample LLM run command, when OLLama is installed: `ollama run phi3:3.8b`

2. Take 1 thousand random passages from the FIQA-PL corpus. INFO: You can play with new dataset, but it will be necessary to create baseline results (next excersise).

In [4]:
from datasets import load_dataset
import pandas as pd
import numpy as np

def modify_df(df, labels):
    for label in labels:
        if label[0] == '_':
            df = df.rename(columns={label: label[1:]})
            label = label[1:]
        df[label] = df[label].astype(np.int64)
    return df

ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
df_corpus = pd.DataFrame(ds['corpus'])
df_corpus = modify_df(df_corpus, ['_id'])
print(df_corpus.head())

ds_q = load_dataset("clarin-knext/fiqa-pl", "queries")
ds_q = pd.DataFrame(ds_q['queries'])
ds_q = modify_df(ds_q, ['_id'])
print(ds_q.head())

ds_qrels = load_dataset("clarin-knext/fiqa-pl-qrels")
df_qrels = pd.DataFrame(ds_qrels['train'])
df_qrels_test = pd.DataFrame(ds_qrels['test'])
df_qrels = modify_df(df_qrels, ['query-id', 'corpus-id', 'score'])
df_qrels_test = modify_df(df_qrels_test, ['query-id', 'corpus-id', 'score'])
print(df_qrels.head())
print(df_qrels_test.head())

   id title                                               text
0   3        Nie mówię, że nie podoba mi się też pomysł szk...
1  31        Tak więc nic nie zapobiega fałszywym ocenom po...
2  56        Nigdy nie możesz korzystać z FSA dla indywidua...
3  59        Samsung stworzył LCD i inne technologie płaski...
4  63        Oto wymagania SEC: Federalne przepisy dotycząc...
   id title                                               text
0   0        Co jest uważane za wydatek służbowy w podróży ...
1   4        Wydatki służbowe - ubezpieczenie samochodu pod...
2   5                        Rozpoczęcie nowego biznesu online
3   6           „Dzień roboczy” i „termin płatności” rachunków
4   7        Nowy właściciel firmy – Jak działają podatki d...
   query-id  corpus-id  score
0         0      18850      1
1         4     196463      1
2         5      69306      1
3         6     560251      1
4         6     188530      1
   query-id  corpus-id  score
0         8     566392      1
1   

In [5]:
df_corpus.loc[0]["text"]

# for text in df_corpus.iterrows():
    # if text[1]['text'].find("Sylvester Stallone") != -1:
        # print(f"found Sylvester Stallone id: {text[1]['id']}")


'Nie mówię, że nie podoba mi się też pomysł szkolenia w miejscu pracy, ale nie możesz oczekiwać, że firma to zrobi. Szkolenie pracowników to nie ich praca – oni tworzą oprogramowanie. Być może systemy edukacyjne w Stanach Zjednoczonych (lub ich studenci) powinny trochę martwić się o zdobycie umiejętności rynkowych w zamian za ich ogromne inwestycje w edukację, zamiast wychodzić z tysiącami zadłużonych studentów i narzekać, że nie są do niczego wykwalifikowani.'

In [6]:
ds_q_mapper = dict()
ds_corpus_mapper = dict()

for idx, query in ds_q.iterrows():
    ds_q_mapper[query['id']] = query['text']

for idx, corpus in df_corpus.iterrows():
    ds_corpus_mapper[corpus['id']] = corpus['text']


print(ds_q_mapper[5993])

Dlaczego ktokolwiek miałby chcieć najpierw spłacić swoje długi w inny sposób niż „najwyższe odsetki”?


In [7]:
import random

corpus_ids = df_corpus["id"].tolist()
hundred_passages = dict()

while len(hundred_passages) < 100:
    # choice_id = random.choice(corpus_ids)
    choice_id = corpus_ids[len(hundred_passages)]
    passage = ds_corpus_mapper[choice_id]
    hundred_passages[choice_id] = passage

In [8]:
passages = list(hundred_passages.items())
print(passages[1])

(31, 'Tak więc nic nie zapobiega fałszywym ocenom poza dodatkową kontrolą ze strony rynku/inwestorów, ale istnieją pewne nowsze kontrole, które uniemożliwiają instytucjom korzystanie z nich. W ramach DFA banki nie mogą już polegać wyłącznie na ratingach kredytowych jako należytej staranności przy zakupie instrumentu finansowego, więc to jest plus. Intencją jest to, że jeśli instytucje finansowe wykonują swoją własną pracę, to *być może* dojdą do wniosku, że określony CDO jest śmieciem, czy nie. Edycja: wprowadzenie')


3. As baseline use traditional NER methods from lab 7 - SpaCy.

In [6]:
import spacy
nlp = spacy.load('pl_core_news_md')
nlp.add_pipe("sentencizer")

classes_freq = dict()
classes_examples = dict(dict())

for passage in hundred_passages.values():
    docs = nlp(passage)
    print(docs.ents)
    for token in docs.ents:
        print(token.lemma_)
        print(token.label_)
    break
    for token in docs.ents:
        if token.label_ in classes_freq:
            classes_freq[token.label_] += 1
        else:
            classes_freq[token.label_] = 1
        if token.label_ in classes_examples:
            if token.lemma_ in classes_examples[token.label_]:
                classes_examples[token.label_][token.lemma_] += 1
            else:
                classes_examples[token.label_][token.lemma_] = 1
        else:
            classes_examples[token.label_] = dict()
            classes_examples[token.label_][token.lemma_] = 1

(Stanach Zjednoczonych,)
Stany Zjednoczone
placeName


In [9]:
import spacy

nlp = spacy.load("pl_core_news_sm")

def transform_text(text):
    doc = nlp(text)

    entities = []
    for ent in doc.ents:
        label = ''
        if ent.label_ == "placeName":
            label = "GPE"
        elif ent.label_ == "geogName":
            label = "GPE"
        elif ent.label_ == "persName":
            label = "PER"
        elif ent.label_ == "orgName":
            label = "ORG"
        elif ent.label_ == "date":
            label = "DATE" 
        elif ent.label_ == "time":
            label = "TIME"
        entities.append((ent.text, label, (ent.start_char, ent.end_char)))
    return entities

def extract_entities(text):
    entities = {
        "place": [],
        "organization": [],
        "date": [],
        "person": [],
        "geographical name": [],
        "time": []
    }

    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "placeName":
            entities['place'].append(ent.text)
        elif ent.label_ == "geogName":
            entities['geographical name'].append(ent.text)
        elif ent.label_ == "persName":
            entities['person'].append(ent.text)
        elif ent.label_ == "orgName":
            entities['organization'].append(ent.text)
        elif ent.label_ == "date":
            entities['date'].append(ent.text)
        elif ent.label_ == "time":
            entities['time'].append(ent.text)

    return entities



In [11]:
import csv

csv_file = pd.read_csv("texts_for_nlp1.csv", delimiter=',')
rows = []
idx = 0
for row in csv_file.iterrows():
    row[1]['Encje'] = transform_text(row[1]['Tekst'])
    print("id: ", idx)
    print("tekst: ", row[1]['Tekst'])
    print("ner: ", row[1]['Encje'])
    print()
    idx += 1
    rows.append([row[1]['Tekst'], row[1]['Encje']])

id:  0
tekst:  Elon Musk mógłby uratować pozbawione poczucia humoru Niemcy przed gospodarczym upadkiem. To nieprawda, że "tylko AfD może uratować Niemcy", jak sugeruje Elon Musk. Jednak reakcje mainstreamu na słowa biznesmena pokazują, jak słuszny jest jego instynkt. Kraj potrzebuje radykalnych zmian. Jeśli w skład następnej koalicji wejdą socjaldemokraci lub Zieloni, Niemcy zostaną spisane na straty.

ner:  [('Elon Musk', 'PER', (0, 9)), ('Niemcy', 'GPE', (53, 59)), ('Niemcy', 'GPE', (131, 137)), ('Elon Musk', 'PER', (153, 162)), ('Zieloni', 'ORG', (347, 354)), ('Niemcy', 'GPE', (356, 362))]

id:  1
tekst:  Marcin Romanowski dostał azyl na Węgrzech. Na decyzję Budapesztu zareagowało polskie Ministerstwo Spraw Zagranicznych. W piątek do resortu wezwano ambasadora Węgier w Polsce, który otrzyma formalny protest. Jarosław Kaczyński przekonuje, że nikt nie prosił Viktora Orbana o pomoc. — On sam tej pomocy udzielił — podkreśla. Z kolei Roman Giertych mówi nam, że strona rządowa "nie ma sp

In [12]:
import re

patterns_for_rows = {
    "0": [('AfD', 'ORG'), ('socjaldemokraci', 'ORG')],
    "1": [('piątek', 'DATE'), ('Faktem', 'ORG')],
    "2": [('Wigilia', 'DATE')],
    "3": [('SportItalia', 'ORG')],
    "4": [],
    "5": [('Kreml', 'GPE')],
    "6": [('Gazeta Wyborcza', 'ORG')],
    "7": [('Ukrainska Prawda', 'ORG')],
    "8": [],
    "9": [],
    "10": [('Dnia Pracowników Służb Bezpieczeństwa', 'DATE')],
    "11": [],
    "12": [('Grenell', 'PER'), ('Trumpowi', 'PER')],
    "13": [],
    "14": [('Gintautasa', 'PER')],
    "15": [('Francuzi', 'PER')],
    "16": [('Wigilię', 'DATE'), ('klubu Polski 2050', 'ORG'), ('X', 'ORG')],
    "17": [('czwartek', 'DATE')],
    "18": [],
    "19": [],
    "20": [('Fakt', 'ORG')],
    "21": [],
    "22": [('World Tennis League', 'ORG'), ('Abu Zabi', 'GPE')],
    "23": [('Baszszara al-Asada', 'PER')],
}

for idx, (_, patterns) in enumerate(patterns_for_rows.items()):
    text = rows[idx][0]
    for pattern in patterns:
        for i in range(len(rows[idx][1])):
            if rows[idx][1][i][0] == pattern[0]:
                rows[idx][1].pop(i)
                break
        match = re.search(pattern[0], text)
        rows[idx][1].append((pattern[0], pattern[1], match.span()))

In [41]:
with open('texts_for_nlp2.csv', 'w', newline='') as csvfile:
    fieldnames = ['Tekst', 'Encje']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()

    for row in rows:
        writer.writerow({"Tekst": row[0], "Encje": row[1]})


4. Design prompts for the LLM to:
   * Identify named entities in text
   * Classify them into predefined categories (person, organization, location, etc.)

In [13]:
prompts = {
    "placeNames": "",
    "orgName": "",
    "date": "",
    "persName": "",
    "geogName": "",
    "time": ""
}

prompts2 = {
    "zero-shot": "Extract named entities from this text and organize them into categories (categories are: place, organization, date, person, geographical name, time) as a Python dictionary with arrays and without any informations from you. Text: ",
    "zero-shot2": "Extract named entities from this text and organize them into categories (categories are: place, organization, date, person, geographical name, time) and list them in concise way, without any additional informations from you. Text:",
    "zero-shot-pl": "Extract named entities from this text and organize them into categories (categories are: place, organization, date, person, geographical name, time) and list them in polish language in a concise way, without any additional informations from you. Text:",
    "few-shot": "Extract named entities from this text and organize them into categories (categories are: place, organization, date, person, geographical name, time) and list them without any additional informations from you like it is showed in next 5 examples:"+
"Tekst: FC Barcelona chce dobrze zakończyć rok. Rywal najtrudniejszy z możliwych. FC Barcelona zmierzy się z Atletico Madryt w meczu 18. kolejki LaLiga, który odbędzie się 21 grudnia 2024 r. o godzinie 21:00 na estadio olimpico Lluis Companys w Barcelonie. Obie drużyny mają po 38 punktów, co czyni to spotkanie kluczowym w kontekście walki o pozycję lidera w tabeli."+
"NER Result: \
        place: Lluis Companys, Barcelonie\
        organization: LaLiga, Atletico Madryt, FC Barcelona\
        date: 21 grudnia 2024 r.,\
        person: none\
        geographical name: none\
        time: godzinie 21:00"+ 
"Tekst: ""Fakt"" wybrał się na jarmark w Gdańsku. Przyjrzeliśmy się ofercie stoisk z jedzeniem i piciem, ale także i tych sprzedających świąteczne ozdoby czy upominki. Na jednym z nich dostrzegliśmy maskotkę stworzoną na wzór ważnej w Polsce osoby. Skierowaliśmy pytanie do zainteresowanego o to, co sądzi o tym pomyśle."+
"NER Result: \
        place: Gdańsku\
        organization: Fakt, Polsce\
        date: none\
        person: none\
        geographical name: none\
        time: none"+ 
"Tekst: Honda i Nissan, dwie z największych japońskich firm motoryzacyjnych, analizują możliwość produkcji pojazdów w swoich zakładach jako część planu pogłębienia współpracy, a potencjalnie nawet połączenia sił. Jak donosi agencja Kyodo, może to oznaczać kolejny krok ku zacieśnieniu relacji między tymi firmami."+
"NER Result: \
        place: none\
        organization: Honda, Nissan, Kyodo\
        date: none\
        person: none\
        geographical name: japońskich\
        time: none"+ 
"Tekst: Iga Świątek wróciła na kort po zawieszeniu za pozytywny wynik testu antydopingowego. Polka wystąpiła w pokazowym turnieju World Tennis League w Abu Zabi. Zaczęła od deblowej porażki, później pokazała się jednak ze znakomitej strony w mecz z Jasmine Paolini."+
"NER Result: \
        place: Abu Zabi\
        organization: World Tennis League\
        date: none\
        person: Iga Świątek, Jasmine Paolini, Polka\
        geographical name: none\
        time: none"+ 
"Tekst: Nagły upadek reżimu Baszszara al-Asada w Syrii otwiera drogę do nowego Bliskiego Wschodu. Droga ta będzie musiała prowadzić przez Arabię Saudyjską. Następca tronu Mohammed bin Salman intensywnie zmienia królestwo nad Zatoką Perską. Arabia Saudyjska niesie dziś ze sobą prawdziwą siłę — i stoi przed kilkoma ogromnymi wyzwaniami."+
"NER Result: \
        place: none\
        organization: Syrii, Arabię Saudyjską\
        date: none\
        person: Baszszara al-Asada, Mohammed bin Salman\
        geographical name: Bliskiego Wschodu, Zatoką Perską\
        time: none"+
"This text is for you:"
"Tekst: "

}


5. Implement prompt variations to compare performance:
   * Zero-shot prompting
   * Few-shot prompting with 3-5 examples

## Zero-shot

In [8]:
import requests
from tqdm import tqdm
import csv

url = "http://localhost:11434/api/generate"
payload = {
    "model": "mistral",
    "prompt": "",
    "stream": False
}

responses = []
idx = 0
for passage in tqdm(hundred_passages.values()):
    idx += 1
    payload["prompt"] = prompts2['zero-shot2']+passage
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            resp = response.json()
            print("Response:", resp)
            responses.append(resp['response'])
            with open('responses2.csv', 'a', newline='') as csvfile:
                fieldnames = ['id', 'Response']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
                writer.writerow({"id": idx, "Response": resp['response']})
        else:
            print(f"Failed with status code: {response.status_code}, Message: {response.text}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [01:25<2:21:32, 85.79s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-23T00:26:41.357710304Z', 'response': '1. Place: Stan Zjednoczone\n\n2. Organization (implied): Firma (unspecified)\n\n3. Person: not mentioned in the text\n\n4. Geographical name: not mentioned in the text\n\n5. Date: not mentioned in the text\n\n6. Time: not mentioned in the text\n\n7. Topic/Concept: szkolenie pracowników (training employees), oprogramowanie (software development), systemy edukacyjne (educational systems), edukacja (education), zadłużeni studenti (indebted students)', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29527, 1180, 1058, 4640, 15296, 29493, 22933, 8756, 4299, 19209, 5960, 7147, 1479, 29650, 140

  2%|▏         | 2/100 [02:13<1:43:39, 63.47s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-23T00:27:29.301788882Z', 'response': ' place: None\n\n   organization: DFA (Derivatives Financial Association), banks\n\n   date: None\n\n   person: None\n\n   geographical name: None\n\n   time: None', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29506, 1259, 1043, 15296, 29485, 16518, 8756, 19828, 1366, 1180, 3272, 4354, 29611, 29481, 2908, 29495, 1850, 1057, 16466, 1068, 2395, 2934, 22984, 1038, 13405, 29634, 22442, 2206, 29634, 7565, 1885, 3325, 1176, 1803, 4675, 29516, 1030, 8239, 1039, 4640, 29493, 12889, 3871, 5023, 12772, 1525, 29495, 1253, 1823, 29481, 2142, 22442, 10374, 29493, 27088, 1289, 4774, 29477, 29650, 1

...

Response: {'model': 'mistral', 'created_at': '2024-12-23T02:52:55.389878435Z', 'response': '1. Place: None\n\n2. Organization: ESOP (Employee Stock Ownership Plan), Google, Firma (Company)\n\n3. Date: None\n\n4. Person: Nieznany (Unknown)\n\n5. Geographical Name: None\n\n6. Time: Wygaśnięcie daty (Expiration date), Nabywanie uprawnień (Purchase schedule)\n\nThe text provided does not contain specific information about people or geographical names, but it discusses different types of ESOP (Employee Stock Ownership Plan), mentions companies like Google and a private/public firm (Firma), and refers to time-related concepts such as expiration dates for options and purchase schedules.', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 147

 99%|█████████▉| 99/100 [2:28:47<01:28, 88.62s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-23T02:54:03.517007445Z', 'response': "1. Form (Formularz 1040, Schedule C, Schedule SE) - organization\n\n2. Country - United States, Great Britain - geographical name\n\n3. Profession - licensed EA/CPA - person\n\n4. Possible issue - tax agreements and totalization agreements between US and UK - date is not explicitly mentioned but it's implied to be related to tax filing which is usually yearly\n\n5. Place of work - Great Britain - geographical name", 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 5593, 29528, 29632, 7704, 18712, 1043, 1339, 29611, 7000, 29611, 18709, 16009, 5023, 3863, 5313, 1989, 1866, 29532, 29493, 19183, 208

100%|██████████| 100/100 [2:33:51<00:00, 92.32s/it] 

Response: {'model': 'mistral', 'created_at': '2024-12-23T02:59:07.219690584Z', 'response': ' Place: N/A\n Organizations: Corporation, Company (specific not given), Fundusze Inwestycyjny (Investment Fund)\n Persons: Not mentioned\n Date: N/A\n Geographical Names: N/A\n Time: Not mentioned', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29655, 2319, 29493, 1769, 1699, 1046, 6640, 28519, 29493, 11340, 1055, 1256, 4935, 1997, 29492, 16674, 8513, 1850, 8481, 2184, 1084, 4198, 1134, 1055, 1256, 29532, 7018, 6818, 9953, 11487, 1454, 1061, 5193, 15500, 10934, 5645, 1043, 2184, 29611, 21489, 16281, 9495, 1184, 29492, 29491, 1119, 4640, 19192, 29485, 6818, 9953, 5023, 29493, 2986, 2965

## Few-shot

In [7]:
import requests
from tqdm import tqdm
import csv

url = "http://localhost:11434/api/generate"
payload = {
    "model": "mistral",
    "prompt": "",
    "stream": False
}

responses = []
idx = 0
for passage in tqdm(hundred_passages.values()):
    idx += 1
    payload["prompt"] = prompts2['few-shot']+passage+"\nNER Results: "
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            resp = response.json()
            print("Response:", resp)
            responses.append(resp['response'])
            with open('responses_few_shot.csv', 'a', newline='') as csvfile:
                fieldnames = ['id', 'Response']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
                writer.writerow({"id": idx, "Response": resp['response']})
        else:
            print(f"Failed with status code: {response.status_code}, Message: {response.text}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)

  1%|          | 1/100 [03:47<6:15:04, 227.32s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-21T14:52:06.294978718Z', 'response': ' place: miejsce pracy (not specified)\n   organization: firma (not specified), Stan Zjednoczone, studenci\n   date: none\n   person: none\n   geographical name: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 2642, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 29551, 29491, 22608, 2953

  2%|▏         | 2/100 [04:31<3:15:45, 119.85s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-21T14:52:51.050041775Z', 'response': ' place: DFA banki\n   organization: rynku/inwestorów\n   date: none\n   person: none\n   geographical name: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 2642, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 29551, 29491, 22608, 29536, 4198, 2468, 29526, 8001, 29493, 19183, 1057, 3452,

  3%|▎         | 3/100 [05:17<2:18:53, 85.91s/it] 

...

Response: {'model': 'mistral', 'created_at': '2024-12-21T17:41:19.220237085Z', 'response': ' place: none\n   organization: Google, plan (emitting stocks or options)\n   date: none\n   person: none\n   geographical name: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 2642, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 29551, 29491, 22608, 29536, 4198, 2468, 29526, 8001, 29493

 99%|█████████▉| 99/100 [2:54:02<01:26, 86.03s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-21T17:42:21.608896428Z', 'response': ' place: none\n   organization: formularz 1040, Schedule C, Schedule SE Formularz “”, Stany Zjednoczone, Wielkiej Brytanii, licencjonowanym EA/CPA\n   date: none\n   person: none\n   geographical name: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 2642, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486,

100%|██████████| 100/100 [3:00:34<00:00, 108.35s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-21T17:48:54.052424221Z', 'response': ' place1: Hedge Fund\n      definition1: A hedge fund is an investment vehicle that pools capital from accredited investors and uses various strategies to manage risk. It is not publicly traded and it can employ complex trading techniques, such as hedging, derivatives, and leveraged and long/short positions in a variety of assets.\n\n   place2: Mutual Fund\n      definition2: A mutual fund is an investment company that pools money from many investors to purchase a diversified portfolio of stocks, bonds, or other securities. It is also not publicly traded, but it must disclose its holdings and trading activities regularly. The mutual fund company manages the portfolio on behalf of the investors.', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106

6. Compare results between:
   * Traditional NER (SpaCy)
   * Pure LLM-based approach

## NER

In [11]:
from tqdm import tqdm
import csv

responses = []
idx = 0
for passage in tqdm(hundred_passages.values()):
    idx += 1
    response = extract_entities(passage)
    resp = response
    with open('responses_ner.csv', 'a', newline='') as csvfile:
        fieldnames = ['id', 'Response']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writerow({"id": idx, "Response": resp})

    responses.append(resp)

100%|██████████| 100/100 [00:04<00:00, 24.22it/s]


## Comparison

In [44]:
from prettytable import PrettyTable 
import textwrap

table_with_results = PrettyTable(['LLM-zero-shot', 'LLM-few-shot', 'NER'])
fieldnames = ['id', 'Response']
responses_zero_shot = open('responses2.csv', 'r', newline='')
reader_zero_shot = csv.DictReader(responses_zero_shot, fieldnames=fieldnames, delimiter=';')
responses_few_shot = open('responses_few_shot.csv', 'r', newline='')
reader_few_shot = csv.DictReader(responses_few_shot, fieldnames=fieldnames, delimiter=';')
responses_ner = open('responses_ner.csv', 'r', newline='')
reader_ner = csv.DictReader(responses_ner, fieldnames=fieldnames, delimiter=';')
cell_length = 50
for i, (res_zero_shot, res_few_shot, res_ner) in enumerate(zip(reader_zero_shot, reader_few_shot, reader_ner)):
    res_zero_shot = textwrap.fill(res_zero_shot['Response'], cell_length)
    res_few_shot = textwrap.fill(res_few_shot['Response'], cell_length)
    res_ner = textwrap.fill(res_ner['Response'], cell_length)
    
    
    table_with_results.add_row([res_zero_shot, res_few_shot, res_ner])
    table_with_results.add_row(["="*cell_length, "="*cell_length, "="*cell_length])

responses_zero_shot.close()
responses_few_shot.close()
responses_ner.close()

print(table_with_results)

+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|                   LLM-zero-shot                    |                    LLM-few-shot                    |                        NER                         |
+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|                      Response                      |                      Response                      |                      Response                      |
| ================================================== | ================================================== | ================================================== |
|    1. Place: Stan Zjednoczone  2. Organization     |        place: miejsce pracy (not specified)        |        {'place': ['Stanach Zjednoczonych'],        |
|   (implied): Firma (unspecified)

Comparison:
- the most concise and precise was traditional NER
- LLM gave too much text and gave irrelevant information that wasn't named entities
- LLM made more mistakes like 'organization: fantasy economy', 'Person: Dotyczy (implied subject)', 
- LLM tried to derive implicit information from text e.g.:time: none (but the implied meaning is "when USA and its allies use them against their own nations")

7. Build a simple evaluation pipeline:
   * Manually annotate 20 passages for ground truth (ideally, share those annotated passages in the group, so everyone have much more than 20)
   * Compute precision, recall, and F1 score for each approach
   * Analyze error patterns and classification mistakes

In [14]:
def precision(TP, FP):
    return TP / (TP+FP)

def recall(TP, FN):
    return TP / (TP + FN)

def f1_score(precision, recall):
    return 2*precision*recall/(precision + recall)


In [42]:
import csv

csv_file = pd.read_csv("manually_annotated.csv", delimiter=',')
manually_annotated = []
idx = 0
for row in csv_file.iterrows():
    print("id: ", idx)
    print("tekst: ", row[1]['Tekst'])
    print("ner: ", row[1]['Encje'])
    print()
    idx += 1
    manually_annotated.append([row[1]['Tekst'], row[1]['Encje']])

id:  0
tekst:  Elon Musk mógłby uratować pozbawione poczucia humoru Niemcy przed gospodarczym upadkiem. To nieprawda, że "tylko AfD może uratować Niemcy", jak sugeruje Elon Musk. Jednak reakcje mainstreamu na słowa biznesmena pokazują, jak słuszny jest jego instynkt. Kraj potrzebuje radykalnych zmian. Jeśli w skład następnej koalicji wejdą socjaldemokraci lub Zieloni, Niemcy zostaną spisane na straty.

ner:  [('Elon Musk', 'PER', (0, 9)), ('Niemcy', 'GPE', (53, 59)), ('Niemcy', 'GPE', (131, 137)), ('Elon Musk', 'PER', (153, 162)), ('Zieloni', 'ORG', (347, 354)), ('Niemcy', 'GPE', (356, 362)), ('AfD', 'ORG', (113, 116)), ('socjaldemokraci', 'ORG', (327, 342))]

id:  1
tekst:  Marcin Romanowski dostał azyl na Węgrzech. Na decyzję Budapesztu zareagowało polskie Ministerstwo Spraw Zagranicznych. W piątek do resortu wezwano ambasadora Węgier w Polsce, który otrzyma formalny protest. Jarosław Kaczyński przekonuje, że nikt nie prosił Viktora Orbana o pomoc. — On sam tej pomocy udzielił — podk

In [17]:
import requests
from tqdm import tqdm
import csv

url = "http://localhost:11434/api/generate"
payload = {
    "model": "mistral",
    "prompt": "",
    "stream": False
}

responses = []
idx = 0
for (passage, ner) in tqdm(manually_annotated):
    idx += 1
    payload["prompt"] = prompts2['zero-shot2']+passage
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            resp = response.json()
            print("Response:", resp)
            responses.append(resp['response'])
            with open('manually_annotated_zero_shot.csv', 'a', newline='') as csvfile:
                fieldnames = ['id', 'Response']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
                writer.writerow({"id": idx, "Response": resp['response']})
        else:
            print(f"Failed with status code: {response.status_code}, Message: {response.text}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)

  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:57<22:04, 57.58s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-23T23:58:46.848342935Z', 'response': " Place: Germany\n   Organization: AfD (Alternative für Deutschland), Elon Musk's Company\n   Person: Elon Musk\n   Geographical name: None\n   Date: Not mentioned in the text\n   Time: Not mentioned in the text", 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 6541, 1034, 4547, 29497, 1058, 15144, 29611, 2171, 5041, 1038, 9021, 29668, 2395, 16611, 1835, 3089, 22456, 29532, 2253, 1283, 18044, 29486, 1186, 4774, 3474, 23517, 1087, 4150, 1118, 1051, 4576, 1850, 1350, 1084, 4198, 1134, 29491, 2559, 8756, 29488, 2275, 2473, 29493, 22933, 1113, 1652, 29482, 5566, 9554, 29525, 4186, 11332, 5041

  8%|▊         | 2/24 [02:36<30:05, 82.09s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:00:26.097762789Z', 'response': '1. Person: Marcin Romanowski, Jarosław Kaczyński, Roman Giertych\n\n   2. Organization: Ministerstwo Spraw Zagranicznych (Polish Ministry of Foreign Affairs), Ambasada Węgier w Polsce (Embassy of Hungary in Poland)\n\n   3. Place: Węgrzech (Hungary), Polska (Poland)\n\n   4. Date or Time: Not mentioned in the text\n\n   5. Geographical Name: Not mentioned in the text', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29523, 7295, 1030, 9093, 17755, 1049, 11174, 1032, 2908, 29482, 2647, 1162, 29632, 1588, 15395, 29491, 7852, 2053, 29492, 29532, 17666, 7863, 8490, 4671, 29486, 1454, 1260, 1

...

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:19:46.188756179Z', 'response': '1. Place: Honda, Nissan (Companies are places in this context)\n\n   2. Organization: Honda, Nissan, Agencja Kyodo (Agency Kyodo is a Japanese news agency)\n\n   3. Event: Analysis, Production, Collaboration, Merger (inferred from the text content)\n\n   4. Time (not explicitly mentioned in the text)', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29537, 14714, 1381, 1186, 1583, 1044, 29493, 11511, 1180, 1454, 14878, 25706, 3053, 2908, 1106, 29431, 29477, 13215, 1307, 6567, 3576, 1463, 29532, 24831, 11085, 29493, 3648, 1231, 21374, 4186, 29650, 1912, 1577, 11451, 1987, 17785, 4400, 239

 96%|█████████▌| 23/24 [22:29<00:46, 46.85s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:20:19.176433782Z', 'response': '1. Person: Iga Świątek\n\n   2. Place: Abu Dhabi\n\n   3. Geographical Name: Poland\n\n   4. Organization: World Tennis League\n\n   5. No specific Date or Time found in the text.', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29505, 3272, 19498, 8154, 29634, 18658, 2193, 29593, 1957, 7673, 2647, 1214, 1187, 2395, 25068, 1265, 3807, 8277, 7438, 2395, 2908, 1652, 1648, 29492, 28537, 1617, 2137, 29486, 1164, 1652, 29483, 23738, 22798, 29491, 3306, 2885, 23964, 29634, 2625, 7673, 1043, 16772, 1709, 25315, 2295, 7018, 29486, 4072, 1088, 8248, 6933, 1043, 25480, 1822, 16843, 29491, 1822, 2

100%|██████████| 24/24 [23:28<00:00, 58.67s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:21:17.29750845Z', 'response': '1. Place: Syria, Arabija Saudyjska (Arabia Saudi)\n\n   2. Organization: Reżim Baszszara al-Asada w Syrii (Regime of Bashar al-Assad in Syria), Królestwo nad Zatoką Perską (Kingdom of the Persian Gulf)\n\n   3. Person: Mohammed bin Salman\n\n   4. Geographical name: Zatoka Perska (Persian Gulf)\n\n   The text does not contain any date or time information.', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 1065, 3846, 1632, 1837, 29493, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 29491, 8147, 29515, 29527, 1125, 11853, 1350, 1538, 29497, 1080, 29650, 1089, 29486, 7595, 29532, 5795, 3691, 1157, 29501, 2966, 2308, 1043, 8049, 1141, 29478, 7725, 29495, 11376, 4251, 29489, 29632, 1279, 1

In [18]:
import requests
from tqdm import tqdm
import csv

url = "http://localhost:11434/api/generate"
payload = {
    "model": "mistral",
    "prompt": "",
    "stream": False
}

responses = []
idx = 0
for (passage, ner) in tqdm(manually_annotated):
    idx += 1
    payload["prompt"] = prompts2['few-shot']+passage+"\nNER Results: "
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            resp = response.json()
            print("Response:", resp)
            responses.append(resp['response'])
            with open('manually_annotated_few_shot.csv', 'a', newline='') as csvfile:
                fieldnames = ['id', 'Response']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
                writer.writerow({"id": idx, "Response": resp['response']})
        else:
            print(f"Failed with status code: {response.status_code}, Message: {response.text}")
    except requests.exceptions.RequestException as e:
        print("An error occurred:", e)

  4%|▍         | 1/24 [03:53<1:29:21, 233.10s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:27:34.733294024Z', 'response': ' place: none\n   organization: Elon Musk, AfD (Alternative for Germany), Niemcy\n   date: none\n   person: Elon Musk\n   geographical name: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 29506, 1718, 1071, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 29551, 29491, 22608, 29536, 4198,

  8%|▊         | 2/24 [04:52<48:01, 130.98s/it]  

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:28:34.223353019Z', 'response': ' place: Budapeszt, Polska (Poland), Węgrzech (Hungary)\n   organization: Ministerstwo Spraw Zagranicznych (Polish Ministry of Foreign Affairs), Węgier (Hungary)\n   date: none\n   person: Marcin Romanowski, Jarosław Kaczyński, Roman Giertych\n   geographical name: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 29506, 1718, 1071, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 29

 12%|█▎        | 3/24 [05:18<29:06, 83.19s/it] 

...

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:44:14.089776905Z', 'response': ' organization: Honda, Nissan, Kyodo\n   geographical name: japońskich', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 29506, 1718, 1071, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 29551, 29491, 22608, 29536, 4198, 2468, 29526, 8001, 29493, 19183, 1057, 3452, 29632, 24581, 7147, 29473, 29518, 29508, 251

 96%|█████████▌| 23/24 [21:10<00:40, 40.81s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:44:51.913004775Z', 'response': ' place: Abu Zabi\n   organization: World Tennis League\n   person: Iga Świątek, Jasmine Paolini, Polka\n   geographical name: none\n   date: none\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 29506, 1718, 1071, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 29551, 29491, 22608, 29536, 4198

100%|██████████| 24/24 [22:01<00:00, 55.06s/it]

Response: {'model': 'mistral', 'created_at': '2024-12-24T00:45:43.172849966Z', 'response': ' place: Syrii, Arabię Saudyjską\n   organization: none\n   date: none\n   person: Baszszara al-Asada, Mohammed bin Salman\n   geographical name: Bliskiego Wschodu, Zatoką Perską\n   time: none', 'done': True, 'done_reason': 'stop', 'context': [3, 29473, 2297, 2875, 5928, 20578, 1245, 1224, 3013, 1072, 26193, 1474, 1546, 13955, 1093, 25656, 1228, 29515, 2401, 29493, 7434, 29493, 4376, 29493, 2106, 29493, 3738, 22391, 1909, 29493, 1495, 29499, 1072, 2042, 1474, 2439, 1475, 5638, 5995, 1465, 1245, 1136, 1505, 1146, 1117, 7410, 1065, 2447, 29473, 29550, 10022, 29515, 29506, 1718, 1071, 29515, 9906, 17603, 1252, 1126, 1279, 2241, 2142, 1454, 7818, 29676, 12254, 29668, 1480, 29497, 29491, 16739, 19172, 14878, 1202, 1322, 5023, 3545, 2908, 1454, 4186, 29650, 1912, 29495, 8470, 29491, 9906, 17603, 20291, 1517, 2908, 7147, 1454, 2562, 1663, 2992, 6079, 1411, 29475, 1043, 1296, 4576, 29486, 29473, 29508, 

In [19]:
from tqdm import tqdm
import csv

responses = []
idx = 0
for (passage, ner) in tqdm(manually_annotated):
    idx += 1
    response = extract_entities(passage)
    resp = response
    with open('manually_annotated_ner.csv', 'a', newline='') as csvfile:
        fieldnames = ['id', 'Response']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writerow({"id": idx, "Response": resp})
    responses.append(resp)

100%|██████████| 24/24 [00:00<00:00, 44.24it/s]


In [48]:
from prettytable import PrettyTable 
import textwrap

table_with_results = PrettyTable(['Manually annotated', 'LLM-zero-shot', 'LLM-few-shot', 'NER'])
fieldnames = ['id', 'Response']
responses_zero_shot = open('manually_annotated_zero_shot.csv', 'r', newline='')
reader_zero_shot = csv.DictReader(responses_zero_shot, fieldnames=fieldnames, delimiter=';')
responses_few_shot = open('manually_annotated_few_shot.csv', 'r', newline='')
reader_few_shot = csv.DictReader(responses_few_shot, fieldnames=fieldnames, delimiter=';')
responses_ner = open('manually_annotated_ner.csv', 'r', newline='')
reader_ner = csv.DictReader(responses_ner, fieldnames=fieldnames, delimiter=';')
cell_length = 50

for i, (annotated, res_zero_shot, res_few_shot, res_ner) in enumerate(zip(manually_annotated, reader_zero_shot, reader_few_shot, reader_ner)):
    if i == 0: continue
    res_zero_shot = textwrap.fill(res_zero_shot['Response'], cell_length)
    res_few_shot = textwrap.fill(res_few_shot['Response'], cell_length)
    res_ner = textwrap.fill(res_ner['Response'], cell_length)
    res_annotated = textwrap.fill(str(annotated[1]), cell_length)

    table_with_results.add_row([res_annotated, res_zero_shot, res_few_shot, res_ner])
    table_with_results.add_row(["="*cell_length, "="*cell_length, "="*cell_length, "="*cell_length])

responses_zero_shot.close()
responses_few_shot.close()
responses_ner.close()

print(table_with_results)

+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|                 Manually annotated                 |                   LLM-zero-shot                    |                    LLM-few-shot                    |                        NER                         |
+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+
|  [('Elon Musk', 'PER', (0, 9)), ('Niemcy', 'GPE',  |  Place: Germany    Organization: AfD (Alternative  |     place: none    organization: Elon Musk, AfD    |     {'place': ['Niemcy', 'Niemcy', 'Niemcy'],      |
|  (53, 59)), ('Niemcy', 'GPE', (131, 137)), ('Elon  |  für Deutschland), Elon Musk's Company    Person:  |  (Alternative for Germany), Niemcy  

In [2]:
false_positive = {
    '0': [0, 1, 1, 0],
    '1': [0, 0, 0, 0],
    '2': [0, 5, 0, 0],
    '3': [0, 3, 0, 0],
    '4': [0, 2, 2, 1],
    '5': [0, 2, 2, 0],
    '6': [0, 1, 0, 0],
    '7': [0, 2, 1, 0],
    '8': [0, 0, 2, 0],
    '9': [0, 1, 2, 0],
    '10': [0, 1, 1, 0],
    '11': [0, 0, 0, 0],
    '12': [0, 0, 2, 2],
    '13': [0, 1, 0, 1],
    '14': [0, 1, 0, 1],
    '15': [0, 1, 1, 0],
    '16': [0, 1, 2, 1],
    '17': [0, 3, 3, 0],
    '18': [0, 2, 2, 0],
    '19': [0, 2, 1, 2],
    '20': [0, 7, 0, 0],
    '21': [0, 6, 0, 2],
    '22': [0, 0, 0, 1],
    '23': [0, 2, 0, 2]
}

true_positive = {
    '0': [8, 4, 4, 3],
    '1': [13, 6, 8, 11],
    '2': [2, 2, 1, 1],
    '3': [7, 3, 4, 6],
    '4': [3, 2, 1, 2],
    '5': [5, 2, 4, 4],
    '6': [8, 5, 6, 7],
    '7': [4, 3, 3, 3],
    '8': [13, 5, 5, 12],
    '9': [9, 6, 7, 9],
    '10': [10, 5, 2, 9],
    '11': [11, 6, 8, 10],
    '12': [6, 3, 2, 4],
    '13': [10, 8, 5, 9],
    '14': [9, 5, 5, 7],
    '15': [6, 3, 3, 4],
    '16': [7, 4, 4, 4],
    '17': [4, 2, 1, 3],
    '18': [2, 2, 2, 2],
    '19': [8, 6, 7, 7],
    '20': [3, 1, 2, 2],
    '21': [4, 3, 4, 1],
    '22': [5, 3, 4, 2],
    '23': [7, 4, 6, 6]
}

In [18]:
from prettytable import PrettyTable 
from operator import add
import numpy as np

table_with_scores = PrettyTable(['Manually annotated', 'LLM-zero-shot', 'LLM-few-shot', 'NER'])
def precision(TP, FP):
    return TP / (TP+FP)

def recall(TP, FN):
    return TP / (TP + FN)

def f1_score(precision, recall):
    return 2*precision*recall/(precision + recall)

sum_tp = [0,0,0,0]
sum_fp = [0,0,0,0]
sum_fn = [0,0,0,0]
for (id_tp, TP), (id_fp, FP) in zip(true_positive.items(), false_positive.items()):
    FN = [ max(0, TP[0]- elem) for elem in TP]
    sum_tp = list(map(add, sum_tp, TP))
    sum_fp = list(map(add, sum_fp, FP))
    sum_fn = list(map(add, sum_fn, FN))

sum_tp = np.array(sum_tp)
sum_fp = np.array(sum_fp)
sum_fn = np.array(sum_fn)
prec = precision(sum_tp, sum_fp)
rec = recall(sum_tp, sum_fn)
f1 = f1_score(prec, rec)

table_with_scores.add_row([f"Precision: {prec[0]}", f"Precision: {prec[1]}", f"Precision: {prec[2]}", f"Precision: {prec[3]}"])
table_with_scores.add_row([f"Recall: {rec[0]}", f"Recall: {rec[1]}", f"Recall: {rec[2]}", f"Recall: {rec[3]}"])
table_with_scores.add_row([f"F1_score: {f1[0]}", f"F1_score: {f1[1]}", f"F1_score: {f1[2]}", f"F1_score: {f1[3]}"])
print(table_with_scores)


+--------------------+-------------------------------+-------------------------------+-------------------------------+
| Manually annotated |         LLM-zero-shot         |          LLM-few-shot         |              NER              |
+--------------------+-------------------------------+-------------------------------+-------------------------------+
|   Precision: 1.0   | Precision: 0.6788321167883211 | Precision: 0.8166666666666667 | Precision: 0.9078014184397163 |
|    Recall: 1.0     |   Recall: 0.5670731707317073  |   Recall: 0.5975609756097561  |   Recall: 0.7804878048780488  |
|   F1_score: 1.0    |  F1_score: 0.6179401993355481 |  F1_score: 0.6901408450704226 |  F1_score: 0.839344262295082  |
+--------------------+-------------------------------+-------------------------------+-------------------------------+


### Disclaimer
Mistral was LLM used in this laboratory.\
False_positive and True_positive were denoted manually because it was the most reasonable way to do it. \
This approach still has some mistakes and results produced by LLM and Spacy's NER weren't perfect.\
Finally, it gave the most accurate results, which was expected also by me.

### ERRORS
#### LLM-zero-shot
- Too much additional and useless information
- adds new categories
- makes mistakes
- doesn't recognize part of genuinely named entities
#### LLM-few-shot
- makes mistakes
- doesn't recognize part of genuinely named entities
#### traditional NER
- makes mistakes (but much less compared to LLM)
- doesn't recognize part of genuinely named entities 

Questions (2 points):

1. How does the performance of LLM-based NER compare to traditional approaches? What are the trade-offs in terms of accuracy, speed, and resource usage?\
    **Traditional approaches are better in the classification of named entities. LLM-based NER takes much more time to compute results which aren't accurate at all and it \
        adds a lot of additional information that is not relevant to results. However, it has also an advantages, because it could derive information from context or could\
        spot some information that is not directly named entity but could have significant information like: "Reżim Baszszara al-Asada" "rząd Donalda Tuska".\
        In addition compared to Spacy's NER, the LLM-based approach in both strategies could recognize "Polski 2050" as an organization, whereas NER from Spacy mistook this as a date.\
        So, sometimes LLM has an advantage where particular words need some context to be understood.**
2. Which prompting strategy proved most effective for NER and classification tasks? Why?\
    **Few-shot strategy produced much better results. They were more concise and precise because LLM knew what was expected. It didn't include additional information like "Fakt (Subject)",\
     "Gdańsk (Location of event)" and rather didn't introduce new categories like "No categories: jeden z nich (one of them), oferta stoisk (offer on stalls)" contrary to zero-shot strategy.**
3. What are the limitations and potential biases of using LLMs for NER and classification?\
    **LLM can mistake some names and produce too much additional text, which rather is not necessary. I tried to suppress that additional information utterly, but I could only to some degree.**
4. In what scenarios would you recommend using traditional NER vs. LLM-based approaches?\
    **In most tasks, traditional NER is recommended, but LLM-based could be better for information that requires context in the text to understand the real meaning and named entity.**